In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-20 20:22:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.34MB/s    in 0.2s    

2021-02-20 20:22:59 (4.34 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProjectCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
# url="https://<bucket name>.s3.amazonaws.com/user_data.csv"
url="https://final-project-g4.s3.amazonaws.com/whr2015to2019.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("whr2015to2019.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show()

+----+-----------+--------------+-----+------------+-------------+---------------------+-------+----------+----------------------+
|Year|Overallrank|       Country|Score|gdppercapita|socialsupport|healthylifeexpectancy|freedom|generosity|perceptionofcorruption|
+----+-----------+--------------+-----+------------+-------------+---------------------+-------+----------+----------------------+
|2019|          1|       Finland|7.769|        1.34|        1.587|                0.986|  0.596|     0.153|                 0.393|
|2019|          2|       Denmark|  7.6|       1.383|        1.573|                0.996|  0.592|     0.252|                  0.41|
|2019|          3|        Norway|7.554|       1.488|        1.582|                1.028|  0.603|     0.271|                 0.341|
|2019|          4|       Iceland|7.494|        1.38|        1.624|                1.026|  0.591|     0.354|                 0.118|
|2019|          5|   Netherlands|7.488|       1.396|        1.522|                0

Postgres Setup

In [ ]:
# Configure settings for RDS:PostgreSQL:PostgreSQL
mode = "append"
jdbc_url="jdbc:postgresql://mypostgres.cl5zvbmd7k4j.us-east-1.rds.amazonaws.com:5432/mydatabase"
config = {"user":"root", 
          "password": "rootroot", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write DataFrame to whr2015to2019 table in PostgreSQL

user_data_df.write.jdbc(url=jdbc_url, table='whr2015to2019', mode=mode, properties=config)